# MATERIAS

In [1]:
# Cada clave es un semestre (columna). El orden sigue la foto.

semestres = {
    1: [
        "Pre Cálculo",
        "Cálculo 1",
        "Matemáticas Discretas",
        "Introducción a la Ciencia de Datos",
        "Programación 1",
        "Instituciones Políticas",
        "Pensamiento Crítico en Ciencia de Datos 1",
        "Idioma 1",
    ],
    2: [
        "Cálculo 2",
        "Álgebra Lineal",
        "Probabilidad",
        "Programación 2",
        "Ética en Ciencia de Datos",
        "Fundamento de Estadística",
        "Idioma 2",
    ],
    3: [
        "Inferencia Estadística",
        "Data Mining",
        "Estructuras de Datos",
        "Introducción a la Economía",
        "Seguridad y Privacidad",
        "Idioma 3",
    ],
    4: [
        "Estadística Bayesiana",
        "Machine Learning 1",
        "Bases de Datos",
        "Disciplina 1",
        "Pensamiento Crítico en Ciencia de Datos 2",
        "Idioma 4",
    ],
    5: [
        "Visualización 1",
        "Métodos Numéricos",
        "Machine Learning 2",
        "Big Data",
        "Disciplina 2",
        "Emprendimiento",
        "Idioma 5",
    ],
    6: [
        "Visualización 2",
        "Optimización",
        "Deep Learning",
        "Formulación de Proyectos",
        "Disciplina 3",
        "Idioma 6",
    ],
    7: [
        "Prácticas",
    ],
    8: [
        "Taller de Habilidades Profesionales",
        "Ciberseguridad",
        "Proyectos",
        "Disciplina 4",
        "Taller de Habilidades Gerenciales",
    ],
}



# CLASE PROYECCIÓN

In [4]:
import math
import pandas as pd

class Proyector:
    # Materias con tope 20 por grupo
    MATERIAS_MAX20 = {
        "Pre Cálculo","Cálculo 1", "Cálculo 2", "Matemáticas Discretas", "Álgebra Lineal",
        "Probabilidad", "Programación 1", "Programación 2", "Inferencia Estadística",
        "Data Mining", "Estructuras de Datos", "Estadística Bayesiana",
        "Machine Learning 1", "Machine Learning 2", "Bases de Datos",
        "Métodos Numéricos", "Big Data", "Optimización", "Deep Learning"
    }

    def __init__(self, redondeo: str | None = None):
        self.redondeo = redondeo
        self._rows: list[pd.DataFrame] = []

    def _pct_to_decimal(self, p):
        p = float(p)
        if p < 0: p = 0.0
        if p > 1: p = p / 100.0
        return p

    def _rnd(self, x):
        if self.redondeo == "round":
            return int(round(float(x)))
        if self.redondeo == "ceil":
            return int(math.ceil(float(x)))
        return float(x)

    def calcular(self, materia: str, porcentaje_perdida, inscritos_actual: float, inscritos_nuevos: float = 0.0) -> pd.DataFrame:
        """
        - 'inscritos_actual' = estudiantes que están concurriendo este semestre.
        - 'inscritos_nuevos' = estudiantes que *van a entrar* (nuevas inscripciones para el siguiente periodo).

        Proyección final (Estudiantes_proyectados) = sobrevivientes_del_semestre_actual + inscritos_nuevos
        donde
            perdieron_actual = inscritos_actual * porcentaje_perdida
            sobrevivientes_del_semestre_actual = inscritos_actual - perdieron_actual

        """
        p = self._pct_to_decimal(porcentaje_perdida)

        # pérdida sobre los inscritos que están concurriendo este semestre
        perdieron_actual = inscritos_actual * p
        sobrevivientes_actual = inscritos_actual - perdieron_actual

        # proyección total = los que permanecen + los nuevos inscritos
        proyectados = perdieron_actual + inscritos_nuevos

        fila = {
            "Materia": materia,
            "Inscrito_actual": self._rnd(inscritos_actual),
            "Inscritos_nuevos": self._rnd(inscritos_nuevos),
            "Perdida_%": round(p * 100, 2),
            "Estudiantes_perdieron_actual": self._rnd(perdieron_actual),
            "Sobrevivientes_actual": self._rnd(sobrevivientes_actual),
            "Estudiantes_proyectados": self._rnd(proyectados),
        }
        return pd.DataFrame([fila])

    def add(self, materia: str, porcentaje_perdida, inscritos_actual: float, inscritos_nuevos: float = 0.0):
        self._rows.append(self.calcular(materia, porcentaje_perdida, inscritos_actual, inscritos_nuevos))
        return self

    def tabla(self) -> pd.DataFrame:
        if not self._rows:
            return pd.DataFrame(columns=[
                "Materia","Inscrito_actual","Inscritos_nuevos","Perdida_%",
                "Estudiantes_perdieron_actual","Sobrevivientes_actual","Estudiantes_proyectados"
            ])
        return pd.concat(self._rows, ignore_index=True)

    def reset(self):
        self._rows.clear()
        return self

    def dividir_grupos(self, df: pd.DataFrame | None = None, cap_otras: int = 35) -> pd.DataFrame:
        """
        Usa la tabla (que debe contener 'Estudiantes_proyectados') y calcula:
        - Capacidad_usada
        - No_grupos
        - Tamaños_grupos
        """
        base = self.tabla() if df is None else df.copy()

        if not {"Materia", "Estudiantes_proyectados"}.issubset(base.columns):
            raise ValueError("Se necesita un DataFrame con columnas 'Materia' y 'Estudiantes_proyectados'.")

        filas = []
        for _, row in base.iterrows():
            materia = str(row["Materia"])
            n = int(round(float(row["Estudiantes_proyectados"] or 0)))

            cap = 20 if materia in self.MATERIAS_MAX20 else int(cap_otras)

            if n <= 0:
                grupos = 0
                sizes = []
            else:
                grupos = math.ceil(n / cap)
                base_size = n // grupos
                extra = n % grupos
                sizes = [base_size + 1] * extra + [base_size] * (grupos - extra)

            filas.append({
                "Materia": materia,
                "Estudiantes_proyectados": n,
                "Capacidad_usada": cap,
                "No_grupos": grupos,
                "Tamaños_grupos": sizes
            })

        out = pd.DataFrame(filas, columns=[
            "Materia", "Estudiantes_proyectados", "Capacidad_usada", "No_grupos", "Tamaños_grupos"
        ])
        return base.merge(out, on=["Materia", "Estudiantes_proyectados"], how="left")


In [7]:
# Ejemplo 
proy = Proyector(redondeo="round")
proy.reset()\
    .add("Matemáticas Discretas", 0.2381, inscritos_actual=82, inscritos_nuevos=100)\
    .add("Cálculo 1", 0.2381, inscritos_actual=19, inscritos_nuevos=100)\
    .add("Programación 1", 20, inscritos_actual=84, inscritos_nuevos=100)\
    .add("Formulación de Proyectos", 0, inscritos_actual=48, inscritos_nuevos=100)\
    .add("Big Data", 0, inscritos_actual=41, inscritos_nuevos=100)\
    .add("Ética", 0, inscritos_actual=61, inscritos_nuevos=100)\
    .add("Optimización", 0.15, inscritos_actual=60, inscritos_nuevos=100)

tabla = proy.tabla()

grupos = proy.dividir_grupos(tabla)
display(grupos)


,Materia,Inscrito_actual,Inscritos_nuevos,Perdida_%,Estudiantes_perdieron_actual,Sobrevivientes_actual,Estudiantes_proyectados,Capacidad_usada,No_grupos,Tamaños_grupos
0,Matemáticas Discretas,82,100,23.81,20,62,120,20,6,"[20, 20, 20, 20, 20, 20]"
1,Cálculo 1,19,100,23.81,5,14,105,20,6,"[18, 18, 18, 17, 17, 17]"
2,Programación 1,84,100,20.00,17,67,117,20,6,"[20, 20, 20, 19, 19, 19]"
3,Formulación de Proyectos,48,100,0.00,0,48,100,35,3,"[34, 33, 33]"
4,Big Data,41,100,0.00,0,41,100,20,5,"[20, 20, 20, 20, 20]"
5,Ética,61,100,0.00,0,61,100,35,3,"[34, 33, 33]"
6,Optimización,60,100,15.00,9,51,109,20,6,"[19, 18, 18, 18, 18, 18]"
